In [1]:
%matplotlib inline
import scanpy as sc
import scrublet as scr
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import gzip

In [2]:
## rewrite 'load_genes' in scrublet to make it compatiable to gzip files
def load_genes(filename, delimiter='\t', column=0, skip_rows=0):
    gene_list = []
    gene_dict = {}

    with gzip.open(filename) as f:
        for iL in range(skip_rows):
            f.readline()
        for l in f:
            gene = l.decode().strip('\n').split(delimiter)[column]
            if gene in gene_dict:
                gene_dict[gene] += 1
                gene_list.append(gene + '__' + str(gene_dict[gene]))
                if gene_dict[gene] == 2:
                    i = gene_list.index(gene)
                    gene_list[i] = gene + '__1'
            else: 
                gene_dict[gene] = 1
                gene_list.append(gene)
    return gene_list

In [3]:
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rc('font', size=14)
plt.rcParams['pdf.fonttype'] = 42

In [4]:
input_dir = '/cndd2/junhao/ALS_FTD_singleCell/run_cellBender_on_raw_snRNA/'

sample_name = [
"MCX_ALS_110",
"MCX_ALS_111",
"MCX_ALS_113",
"MCX_ALS_332",
"MCX_ALS_388",
"MCX_ALS_52",
"MCX_Control_1069",
"MCX_Control_902",
"MCX_Control_904",
"MCX_Control_906",
"MCX_Control_91",
"MCX_Control_945",
"MCX_FTD_36",
"MCX_FTD_54",
"MCX_FTD_55",
"MCX_FTD_61",
"MCX_FTD_674",
"MCX_FTD_908",
"mFCX_ALS_110",
"mFCX_ALS_111",
"mFCX_ALS_113",
"mFCX_ALS_332",
"mFCX_ALS_388",
"mFCX_ALS_52",
"mFCX_Control_1069",
"mFCX_Control_902",
"mFCX_Control_904",
"mFCX_Control_906",
"mFCX_Control_91",
"mFCX_Control_945",
"mFCX_FTD_36",
# "mFCX_FTD_54", # removed because cellBender failed to create h5 file (no cells left)
"mFCX_FTD_55",
"mFCX_FTD_61",
"mFCX_FTD_674",
"mFCX_FTD_908",
]


for selected_sample in sample_name:
    
    adata = sc.read_10x_h5(input_dir + selected_sample + '_cellBender_corrected_filtered.h5')
    adata.var_names_make_unique()
    
    counts_matrix = adata.X.tocsc()
    genes = adata.var_names.values
    barcode = adata.obs_names.values
    barcode = np.array([selected_sample + '_' + item.split('-')[0] for item in barcode])

    print('Counts matrix shape: {} rows(cells), {} columns(genes)'.format(counts_matrix.shape[0], counts_matrix.shape[1]))

    scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)

    doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                              min_cells=3, 
                                                              min_gene_variability_pctl=85, 
                                                              n_prin_comps=30)
    
    # manually set the threshold to call doublet to 0.2
    scrub.call_doublets(threshold=0.2)

    df = pd.DataFrame()
    df['barcode'] = barcode
    df['doublet_scores'] = scrub.doublet_scores_obs_
    df['predicted_doublets'] = scrub.predicted_doublets_
    
    df.to_csv('{}_doublet_score_callThreshold_0.2.tsv'.format(selected_sample), sep="\t", header=True, index=False)
    
    fig, axs=scrub.plot_histogram()
    fig.savefig('{}_doublet_score_hist_callThreshold_0.2.pdf'.format(selected_sample))
    plt.close()

    print('Running UMAP...')
    scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
    print('Done.')
    
    fig, axs=scrub.plot_embedding('UMAP', order_points=True)
    fig.savefig('{}_doublet_score_umap_callThreshold_0.2.pdf'.format(selected_sample))
    plt.close()

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Counts matrix shape: 6931 rows(cells), 33538 columns(genes)
Preprocessing...


/cndd/junhao/anaconda3/envs/scanpy_scrublet/lib/python3.7/site-packages/scrublet/helper_functions.py:321: RuntimeWarning: divide by zero encountered in true_divide
  w.setdiag(float(target_total) / tots_use)


Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.23
Detected doublet rate = 3.0%
Estimated detectable doublet fraction = 37.7%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 8.0%
Elapsed time: 22.9 seconds
Detected doublet rate = 3.4%
Estimated detectable doublet fraction = 40.9%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 8.2%
Running UMAP...
Done.


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


...

Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.50
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 11.8%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 0.3%
Elapsed time: 6.1 seconds
Detected doublet rate = 1.8%
Estimated detectable doublet fraction = 26.2%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 7.0%
Running UMAP...
Done.


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Counts matrix shape: 6589 rows(cells), 33538 columns(genes)
Preprocessing...


/cndd/junhao/anaconda3/envs/scanpy_scrublet/lib/python3.7/site-packages/scrublet/helper_functions.py:321: RuntimeWarning: divide by zero encountered in true_divide
  w.setdiag(float(target_total) / tots_use)


Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.25
Detected doublet rate = 2.2%
Estimated detectable doublet fraction = 39.8%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 5.5%
Elapsed time: 14.4 seconds
Detected doublet rate = 2.9%
Estimated detectable doublet fraction = 45.6%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 6.3%
Running UMAP...
Done.
